In [1]:
from twitter.search import Search
import datetime

In [ ]:
query = 'americanas lang:pt since:2023-09-25'

In [ ]:
search = Search(accounts_json_path="example_accounts.json",
                collection_limit_per_account=500,
                hours_to_reset_collection=12,
                debug=True
                )

latest_results = search.run(
    limit=10000,
    latest=True,
    retries=3,
    queries=[
        {
            'category': 'Latest',
            'query': query
        },
    ],
)

df = search.get_tweets_dataframe()

In [ ]:
df.to_excel(f"{query[:15].replace(' ', '-')}-{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}.xlsx")

In [ ]:
df